In [93]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd

In [95]:
import sys
sys.path.append('./models') 
from gaussian import GP
from gaussian import RBF
from bandit import Bandit

sys.path.append('./train_data') 
from train_func import TrueFunc

In [96]:
# func_sigma_list = [i/10 for i in range(1, 6,1)]
# func_max_list  = [i/10 for i in range(2, 12,2)]

# gp_noise_list   = [i/100 for i in range(5, 35, 5)]
# rbf_alpha_list  = [i/100 for i in range(5, 35, 5)]
# rbf_beta_list   = [i/100 for i in range(5, 35, 5)] 

In [97]:
GP_ME = 0.5
n_play = 100
n_sample = 100

func_sigma_list = [i/10 for i in range(1, 4,1)] #[0.1, 0.2, 0.3]
func_max_list   = [i/10 for i in range(2, 8,2)] #[0.2, 0.4, 0.6]

gp_noise_list   = [i/100 for i in range(5, 35, 10)] #[0.05, 0.15, 0.25]
rbf_alpha_list  = [i/100 for i in range(5, 35, 10)]
rbf_beta_list   = [i/100 for i in range(5, 35, 10)] 

In [98]:
len(func_max_list)*len(func_sigma_list)*len(rbf_beta_list)**3

rbf_beta_list

[0.05, 0.15, 0.25]

In [111]:
def get_max_seed():
    try:
        d = pd.read_csv('../results/csv/gp_ts/gp_ts.csv')
        max_seed = d['seed'].tail(1).values[0]
        return  max_seed
    except FileNotFoundError:
        return 0

In [112]:
def play_bandit_gp_ts():
    
    dir_path = '../results/csv/gp_ts/'
    column   =['f_sigma', 'f_max', 'noise', 'alpha','beta','seed', 'play_time', 'select_arm', 'reward', ]
    column2  =['f_sigma', 'f_max', 'noise', 'alpha','beta', 'total_reward', 'seed' ]
    
    data  = []
    data2 = []
    seed     = get_max_seed() + 1
    
    _df = pd.read_csv('../results/csv/gp_ts/gp_ts_2.csv')
    _df2 = pd.read_csv('../results/csv/gp_ts/gp_ts_2.csv')
    
    for train_func_sigma in func_sigma_list:
        for train_func_max in func_max_list:
            for gp_noise in gp_noise_list:
                for rbf_alpha in rbf_alpha_list:
                    for rbf_beta in rbf_beta_list:
                        train_func = TrueFunc(train_func_sigma, train_func_max).peak_one
                        kernel = RBF(rbf_beta, rbf_alpha)
                        model = GP(GP_ME, gp_noise, kernel)

                        t_r = 0
                        np.random.seed(seed)
                        bandit = Bandit(n_play, n_sample)

                        for play_time in range(n_play):
                            train, reward = bandit.gp_ts_e(train_func, model)
                            t_r += reward
                            data.append([train_func_sigma, train_func_max, gp_noise, rbf_alpha, rbf_beta, seed, play_time, train, reward])
                        data2.append([train_func_sigma, train_func_max, gp_noise, rbf_alpha, rbf_beta, t_r, seed])
                        
                            
    df = pd.DataFrame(data, columns=column)
    df2 = pd.DataFrame(data2, columns=column2)
    
    df = pd.concat([_df, df], ignore_index=True)
    df2 = pd.concat([_df2, df2], ignore_index=True)
    
    csv_name = "gp_ts.csv"
    csv_name2 = "gp_ts_2.csv"
    
    df.to_csv(str(dir_path) + str(csv_name), index=False)
    df2.to_csv(str(dir_path) + str(csv_name2), index=False)

    print(f'finished seed is {seed}')

In [110]:
n = 1
for _ in range(n):
    play_bandit_gp_ts()

finished seed is 2
